## Training process for ECD507 Senior Capstone Project - F1Tenth ML Based Autonomous Race Car
### Contributors - Charles Hodgins, Rishabh Hegde, Dylan DiGiacomo, and Andrew Meccariello

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import ast

In [3]:
print(torch.cuda.is_available())
print(torch.version.cuda)

True
12.6


In [ ]:
#index from around 100,000 for object avoidance

class DrivingDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_path = self.data.iloc[idx,1]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (200, 66))  # Resize using OpenCV
        image = image / 255.0  # Normalize pixel values to [0,1]
        image = np.transpose(image, (2, 0, 1))  # Change shape to (C, H, W)
        image = torch.tensor(image, dtype=torch.float32)
        steering_angle = float(self.data.iloc[idx, 3])
        throttle = float(self.data.iloc[idx, 4])
        lidar = torch.tensor(ast.literal_eval(self.data.iloc[idx, 2]))
               
        return image, lidar, torch.tensor([steering_angle, throttle], dtype=torch.float32)


In [5]:
df = pd.read_csv('/media/ecd507/JetsonOrinNano/home/ecd507/training/data/driving_log.csv')
print(len(df))

112613


/home/ecd507/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3579: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:

def filter_chunk(df):
    global first_chunk
    print(first_chunk)
    print(f"Initial size: {len(df)}")

    def is_valid_image(img_path):
        return os.path.exists(img_path) and cv2.imread(img_path) is not None
    
    df = df.loc[:, ~df.columns.str.contains("^Unnamed")]

    before_throttle = len(df)
    df = df[df["throttle"] != 0 ]
    print(f"Filtered throttle: {before_throttle - len(df)} rows removed")

    before_lidar = len(df)
    df = df[df["lidar_list"] != ""]
    df = df[pd.notna(df["lidar_list"])]
    print(f"Filtered lidar: {before_lidar - len(df)} rows removed")

    before_images = len(df)
    df["image_path"] = df["image_path"].str.strip()
    df = df[df["image_path"] != ""]
    df_filtered = df[df["image_path"].apply(is_valid_image)]
    print(f"Filtered images: {before_images - len(df_filtered)} rows removed")

    print(f"Final size: {len(df_filtered)}")
    
    print(len(df_filtered))
    
    if first_chunk:
        df_filtered.to_csv("/media/ecd507/JetsonOrinNano/home/ecd507/training/data/driving_log2.csv", mode='w', index=False, header=True)
        first_chunk = False
    else:
        df_filtered.to_csv("/media/ecd507/JetsonOrinNano/home/ecd507/training/data/driving_log2.csv", mode='a', index=False, header=False)


    # Save the cleaned dataset


    
first_chunk = True

for chunk in pd.read_csv("/media/ecd507/JetsonOrinNano/home/ecd507/training/data/driving_log.csv",chunksize = 10000):
    print('loaded chunk')
    filter_chunk(chunk)
    # print(chunk.head())
    del chunk
    
print('done')

loaded chunk
True
Initial size: 10000
Filtered throttle: 1009 rows removed
Filtered lidar: 12 rows removed
Filtered images: 0 rows removed
Final size: 8979
8979
loaded chunk
False
Initial size: 10000
Filtered throttle: 1331 rows removed
Filtered lidar: 0 rows removed
Filtered images: 0 rows removed
Final size: 8669
8669
loaded chunk
False
Initial size: 10000
Filtered throttle: 371 rows removed
Filtered lidar: 0 rows removed
Filtered images: 0 rows removed
Final size: 9629
9629
loaded chunk
False
Initial size: 10000
Filtered throttle: 773 rows removed
Filtered lidar: 0 rows removed
Filtered images: 0 rows removed
Final size: 9227
9227
loaded chunk
False
Initial size: 10000
Filtered throttle: 2644 rows removed
Filtered lidar: 0 rows removed
Filtered images: 0 rows removed
Final size: 7356
7356
loaded chunk
False
Initial size: 10000
Filtered throttle: 1082 rows removed
Filtered lidar: 0 rows removed
Filtered images: 0 rows removed
Final size: 8918
8918
loaded chunk
False
Initial size: 523

In [8]:

dataset = DrivingDataset('/media/ecd507/JetsonOrinNano/home/ecd507/training/data/driving_log2.csv')
print(len(dataset))

dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

57160


In [ ]:
class LidarNet(nn.Module):
    def __init__(self):
        super(LidarNet, self).__init__()
        # expects input size of 721
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=24, kernel_size=10, stride=4) # output: (721-10)/4 + 1 = 178
        self.conv2 = nn.Conv1d(in_channels=24, out_channels=36, kernel_size=8, stride=4) # (178-8)/4 + 1 = 43
        self.conv3 = nn.Conv1d(in_channels=36, out_channels=48, kernel_size=4, stride=2) # (43-4)/2 + 1 = 20
        self.conv4 = nn.Conv1d(in_channels=48, out_channels=64, kernel_size=3, stride=1) # (20 -3)/1 + 1 = 1
        self.conv5 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, stride=1) # (18 - 3)/1 + 1 = 16

        self.fc1 = nn.Linear(1024, 512)  # (64 * 16) = 1024 -> 512
        self.fc2 = nn.Linear(512, 256) # 512 -> 256
        self.fc3 = nn.Linear(256, 100) # 256 -> 200
        self.fc4 = nn.Linear(100, 10) # 100 -> 10
        self.fc_out = nn.Linear(10, 2)  # 10 -> 2 Steering Angle & Speed

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = torch.relu(self.conv4(x))
        x = torch.relu(self.conv5(x))
        x = x.view(x.size(0), -1)  # Flatten to (batch, 1024)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc_out(x)  # No activation for raw output
        return x


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # change to cuda when it begins to work
print(device)
model = LidarNet().to(device)
criterion = nn.SmoothL1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001, )

# Training loop
epochs = 10
for epoch in range(epochs):
    total_loss = 0
    for images,lidar,targets in dataloader:
        lidar = lidar.unsqueeze(1)
        lidar, targets = lidar.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(lidar)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader)}')

# Save model
torch.save(model.state_dict(), '/media/ecd507/JetsonOinNano/home/ecd507/training/lidar_model.pth')
print("Model training complete and saved as lidar_model.pth")